In [ ]:
import matplotlib.pyplot as plt;
import numpy as np;
import scipy as scp;
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [ ]:
data = np.loadtxt("../plot_finer/test_with_tof.txt", delimiter=',', dtype=np.complex64);

In [ ]:
data_subtracted=data[:, 4:]
np.shape(data_subtracted)

In [ ]:
plt.imshow(np.square(np.abs(data_subtracted)).T, cmap = "magma", extent = [ data[0,2],data[-1,2], -16/2,16/2 ,],origin="lower",aspect = "auto")
plt.grid(False)
plt.title("Momentum")
plt.xlabel("$t\omega_r$")
plt.ylabel("$p(\hbar k_L)$")

## FFT STYLE 1

In [ ]:
myfft = np.array([scp.fft.fft( data_subtracted[n,:]) for n in range(len(data_subtracted))])

In [ ]:
x_val=np.square(np.abs(myfft))

In [ ]:
plt.imshow(x_val.T, cmap="magma", aspect="auto")

In [ ]:
halfway = int((len(x_val[:,0])-1)/2)
halfway

In [ ]:
x_reshaped = np.concatenate( (x_val[:,halfway:], x_val[:,0:halfway+1]), axis=1 )

In [ ]:
plt.imshow(x_val[:, halfway:-1], cmap="magma", aspect="auto")

In [ ]:
plt.imshow(x_val[:,0:halfway+1], aspect = "auto")

## FFT STYLE 2

In [ ]:
halfway = int((len(data_subtracted[0,:])-1)/2)
halfway
p_reshaped = np.concatenate( (data_subtracted[:,halfway:], data_subtracted[:,0:halfway+1]), axis=1 )

In [ ]:
myfft = np.array([scp.fft.fft(p_reshaped[n]) for n in range(len(data_subtracted))])

In [ ]:
myfft_reshaped=np.concatenate( (myfft[:,halfway:], myfft[:,0:halfway+1]), axis=1 )

## WARNING: Why aren't these normalized

In [ ]:
temp = np.square(np.abs(myfft_reshaped));
myfft_reshaped_rescaled = np.array ( [ temp[m,:]/np.sum(temp[m,:])    for m in range(len(myfft_reshaped)) ] )

In [ ]:
plt.figure(figsize = (12, 6))

plt.subplot(121)
im=plt.imshow(myfft_reshaped_rescaled.T, cmap="hot",aspect="0.04", vmax = 0.03, extent=[data[0,2],data[-1,2], -96, 96])
ax = plt.gca()
#divider = make_axes_locatable(ax)
#cax = divider.append_axes("right", size="25%", pad=0.1)
#plt.colorbar(im, cax=cax)
plt.colorbar(shrink = 0.5)

plt.xlabel("$t\omega_r$")
plt.ylabel("$x (k_L^{-1})$ \n(double check range of $x$)")
plt.title("Wavepacket evolution")
print( 32*np.pi/11.5)
y = np.arange(-96,96,0.1);
plt.plot(np.ones(len(y))*32*np.pi/11.5, y)


plt.subplot(122)
plt.plot(data[:,2],data[:,3], color= "red")
plt.title("DSP shaking with TOF")
plt.ylabel("Amplitude $A$ of $\phi = A\sin\omega t$") 
plt.xlabel("$t(\omega_R^{-1})$")
y2 = np.arange(-1.5*np.pi,1.5*np.pi,0.1);
plt.plot(np.ones(len(y2))*32*np.pi/11.5, y2)
plt.tight_layout()

In [ ]:
time = data[:,2];
dt = time[2]-time[1];
time_to_plot = 17.5 ; # im omega_R inverse
time_index_to_plot = int(18/dt)

fig = plt.figure(figsize=(12,5));
# animation function. This is called sequentially

for t_ind in range( time_index_to_plot, len(time), 100):
    plt.plot( myfft_reshaped_rescaled[t_ind, :])
    plt.pause(0.01)

plt.show()


In [ ]:
time_index_to_plot